# Quick testing:

# Get product URLS and basic details:

In [1]:
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from random import randint
from time import sleep, ctime


def accept_cookies(wd):
    accept_cookies = wd.find_element_by_xpath('/html/body/div/div[1]/div/a')
    accept_cookies.click()


def get_waterstones_links(web_driver, url, start, end):
    """Iterates through waterstones thumbnail pages to get each product URL"""
    errors = []
    print('START:', ctime())
    product_info = []

    while start <= end:
        page = url + str(start)

        try:
            web_driver.get(page)
            products = wd.find_elements_by_class_name('info-wrap') # search result list

            for product in products:
                info = {}

                info['author'] = product.find_element_by_class_name('author').text
                info['price'] = product.find_element_by_class_name('price').text
                info['book_format'] = product.find_elements_by_class_name('format')[-1].text
                info['product_link'] = product.find_element_by_tag_name('a').get_property('href')
                
                product_info.append(info)

            start += 1

        except:
            print(f'Failed on this URL: {page}')
            start += 1
            errors.append(page)
    
    df = pd.DataFrame(product_info)
    df.to_pickle(f'product_links_{start-1}')

    print('END:', ctime())


wd = webdriver.Chrome(ChromeDriverManager().install())
wd.get('https://www.waterstones.com/books/search/')

accept_cookies(wd)
get_waterstones_links(wd, 'https://www.waterstones.com/books/search/page/', 16000, 20834) # max 20834

wd.close()

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\xiaod\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache
 
START: Tue Feb  2 23:31:02 2021
END: Wed Feb  3 06:54:29 2021


In [ ]:
len(product_info)

# Get extra details:

In [6]:
import pandas as pd
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
from time import sleep

links = pd.read_pickle('product_links_2')
links = links.product_link.tolist()

books = []
wd = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://www.waterstones.com/book/doctor-who-sands-of-time/justin-richards/9781849907675'
wd.get(url)



for link in tqdm(links[:2]):
    print(link)
    wd.get(link)


    title = wd.find_element_by_class_name('book-title').text # //*[@id="scope_book_title"]
    publisher = wd.find_element_by_xpath("//span[@itemprop='publisher']").text
    ISBN = wd.find_element_by_xpath("//span[@itemprop='isbn']").text
    categories = wd.find_element_by_xpath("//*[@class='breadcrumbs span12']").text
    full_star = wd.find_elements_by_xpath("//span[@class='star-icon full']")
    half_star = wd.find_elements_by_xpath("//span[@class='star-icon half']")
    num_of_stars = len(full_star) + len(half_star)*0.5
    number_of_reviews = wd.find_elements_by_xpath("//a[@class='reviews-trigger']")
    pages = wd.find_elements_by_xpath("//span[@itemprop='numberOfPages']")
    weight = wd.find_elements_by_xpath("//span[@itemprop='weight']")
    desc = wd.find_elements_by_xpath("//*[@itemprop='description']")[0].text


    extra_information = {'booktitle':title,
        'publisher':publisher,
        'ISBN':ISBN,
        'category':categories,
        'num_of_stars':num_of_stars,
        'num_of_reviews':number_of_reviews,
        'pages':pages,
        'weight':weight,
        'desc':desc}

    books.append(extra_information)

print(books)



[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
 
[WDM] - Driver [C:\Users\xiaod\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache
 50%|█████     | 1/2 [00:00<00:00,  1.07it/s]0
0
0
https://www.waterstones.com/book/venemous-animals/peterson/9780395936085
100%|██████████| 2/2 [00:01<00:00,  1.15it/s]0
0
0
[{'booktitle': 'LGB Elmo Loves You (Sesame Street) (Hardback)', 'publisher': '', 'ISBN': '', 'category': "This book can be found in:\nChildren's & Teenage > Picture books > Characters", 'num_of_stars': 0.0, 'num_of_reviews': [], 'pages': [<selenium.webdriver.remote.webelement.WebElement (session="17adfa14296080e460fa793287650374", element="a52527bc-ca1f-4248-9435-f3f0662e0d8c")>, <selenium.webdriver.remote.webelement.WebElement (session="17adfa14296080e460fa793287650374", element="d62af1b1-da79-4303-b09b-021e1a6603ac")>, <selenium.webdriver.remote.webelement.WebElement (session="17adfa14296080e460fa793287650

In [10]:
import pandas as pd

# 
first = pd.read_pickle('product_links_first')
second = pd.read_pickle('product_links_second')
third = pd.read_pickle('product_links_third')

# 
dfs = [first, second, third]
df = pd.concat(dfs, ignore_index=True)

df.to_csv('all_info.csv',index=False)


In [11]:
df1 = pd.read_csv('all_info', nrows=500048)

In [12]:
#rename columns with title captial
df1.rename(columns={"author": "Author", "price": "Price","book_format":"Book_format","product_link":"Product_link"},inplace=True)

In [29]:
df1['ISBN'] = df1['Product_link'].str.split('/').apply(lambda row: row[-1])
df1['Title'] = df1['Product_link'].str.split('/').apply(lambda row: row[4]).replace('-',' ')
df1['Contributors'] = df1['Product_link'].str.split('/').apply(lambda row: row[5:-1])
#split links into different elements

In [53]:
#drop the £ 
df1['Price'] = df1['Price'].str.strip("£ ")
# convert it to float, for futher analysis
df1['Price'] = df1['Price'].astype('float64')

AttributeError: Can only use .str accessor with string values!

In [44]:
df1['Title'] = df1['Title'].str.replace('-',' ')

In [45]:
df2 = df1[[ 'Title','Author','Contributors','Price','Book_format','ISBN','Product_link']].copy()
df2

,Title,Author,Contributors,Price,Book_format,ISBN,Product_link
0,lgb elmo loves you sesame street,Sarah Albee,[sarah-albee],3.99,Hardback,9780385372831,https://www.waterstones.com/book/lgb-elmo-love...
1,venemous animals,Peterson,[peterson],11.99,Paperback,9780395936085,https://www.waterstones.com/book/venemous-anim...
2,the little butterfly step into reading lvl 2,Sherry Shahan,[sherry-shahan],3.99,Paperback,9780385371896,https://www.waterstones.com/book/the-little-bu...
3,a field guide to the birds nests,Hal H. Harrison,[hal-h-harrison],11.99,Paperback,9780395936092,https://www.waterstones.com/book/a-field-guide...
4,hog eye,Susan Meddaugh,[susan-meddaugh],3.99,Paperback,9780395937464,https://www.waterstones.com/book/hog-eye/susan...
...,...,...,...,...,...,...,...
500043,may gibbs,Robert Holden,"[robert-holden, jane-brummitt]",25.00,Hardback,9781742701509,https://www.waterstones.com/book/may-gibbs/rob...
500044,commentaries on the laws of england of public ...,Knight William Blackstone,[knight-william-blackstone],15.45,Paperback,9781594625558,https://www.waterstones.com/book/commentaries-...
500045,mr asgills apology for an omission in his late...,John Asgill,[john-asgill],34.95,Paperback,9785518817418,https://www.waterstones.com/book/mr-asgills-ap...
500046,marxs concept of man,Erich Fromm,"[erich-fromm, t-bottomore]",9.00,Paperback,9781614270218,https://www.waterstones.com/book/marxs-concept...


In [36]:
df2['ISBN'] = df['ISBN'].astype('int64')

In [37]:
df2.to_csv('book_info.csv')

In [38]:
df2.shape

(500048, 7)